# Column Transformer

In [1]:
import numpy as np
import pandas as pd

In [2]:
# for missing data better to use simple imputer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [6]:
data = pd.read_csv('corona_test.csv')
data.sample(5)

,age,gender,fever,cough,city,has_covid
46,19,Female,101.0,Mild,Mumbai,No
48,66,Male,99.0,Strong,Bangalore,No
62,56,Female,104.0,Strong,Bangalore,Yes
80,14,Female,99.0,Mild,Mumbai,Yes
74,34,Female,104.0,Strong,Delhi,No


In [7]:
data.info()

<class 'pandas.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    str    
 2   fever      90 non-null     float64
 3   cough      100 non-null    str    
 4   city       100 non-null    str    
 5   has_covid  100 non-null    str    
dtypes: float64(1), int64(1), str(4)
memory usage: 4.8 KB


In [9]:
data.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data.drop('has_covid',axis=1), data.has_covid, test_size=0.2, random_state=42)

#### Without using Column transformer

In [13]:
#simple imputer
si= SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])


In [15]:
#ordinal encoding
oe= OrdinalEncoder( categories =[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])


In [24]:
#OneHot encoding

ohe = OneHotEncoder(drop='first',sparse_output = False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])


In [31]:
#extracting age

X_train_age = X_train[['age']].values
X_test_age = X_test[['age']].values
X_train_age.shape

(80, 1)

In [36]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis =1)
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis =1)
X_train_transformed

array([[ 81., 101.,   0.,   0.,   0.,   1.,   0.],
       [  5., 100.,   0.,   0.,   1.,   0.,   0.],
       [ 19., 100.,   0.,   0.,   1.,   0.,   0.],
       [ 27., 100.,   1.,   1.,   0.,   0.,   0.],
       [ 73., 103.,   0.,   1.,   0.,   0.,   0.],
       [ 70., 103.,   1.,   0.,   1.,   0.,   1.],
       [ 49., 102.,   0.,   1.,   0.,   0.,   0.],
       [ 51., 101.,   0.,   0.,   1.,   0.,   1.],
       [ 64., 101.,   0.,   1.,   0.,   0.,   0.],
       [ 83., 101.,   0.,   0.,   1.,   0.,   0.],
       [ 65.,  98.,   0.,   0.,   0.,   1.,   0.],
       [ 18., 104.,   0.,   0.,   0.,   0.,   0.],
       [ 16., 103.,   0.,   0.,   0.,   0.,   0.],
       [ 16., 104.,   1.,   0.,   1.,   0.,   0.],
       [ 27., 100.,   1.,   0.,   1.,   0.,   0.],
       [ 84., 101.,   0.,   0.,   0.,   0.,   0.],
       [ 51., 104.,   1.,   0.,   1.,   0.,   0.],
       [ 69., 102.,   0.,   0.,   0.,   0.,   0.],
       [ 82., 102.,   0.,   0.,   0.,   0.,   1.],
       [ 69., 103.,   0.,   0.,

#### Using Column transformer

In [44]:
from sklearn.compose import ColumnTransformer

#transformer in tuple
transformer= ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(),['fever']),
    ('tnf2', OrdinalEncoder(categories = [['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first', sparse_output=False),['gender','city'])
], remainder='passthrough')

In [50]:
transformer.fit_transform(X_train).shape

(80, 7)

In [49]:
transformer.fit_transform(X_test).shape

(20, 7)